In [74]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

In [75]:
# Constants
pace_threshold = 0.5
#euclidean_distance_threshold = 200

In [76]:
# Read in Split Timings
df = pd.read_csv('../../data/1km_splits_full.csv', encoding = "ISO-8859-1")

In [77]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'bib','team', 'rank', 'nationality'], axis = 1)

In [78]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Male,Senior Men (23-39),5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Senior Men (23-39),5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2017,0d928f7482046900e19dba217d7befa5,Male,Senior Men (23-39),5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,2017,93f20063908ba5d48724dcf31740a52a,Male,Senior Men (23-39),5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,2017,7479fb90beffad048c5f772ba1915b4e,Male,Senior Men (23-39),5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


In [79]:
df.groupby('category').count()

,Year,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),7881,7881,7881,5994,560,954,5990,801,7749,7435,7379,7389,7116,7098,6886,6834,6869,1697,816,5475,6215,6176,5779,5764,5339,1816,540,2726,1811,534,2722,4254,714,4262,714,3707,524,712,3705,523,711,4408,4931,4930,7877
Masters Men 2 (50-59),3393,3393,3393,2592,220,376,2588,373,3312,3145,3116,3119,2966,2944,2848,2814,2838,675,330,2172,2462,2438,2230,2220,2004,736,183,974,733,183,970,1608,239,1606,239,1403,173,235,1398,172,236,1633,1805,1805,3388
Masters Men 3 (60-69),520,520,520,371,43,61,370,74,502,458,453,454,423,417,388,387,384,70,35,281,309,304,261,254,216,100,23,76,100,23,77,180,13,178,13,148,22,13,148,21,13,162,183,183,520
Masters Men 4 (70-79),29,29,29,23,1,4,23,2,28,27,25,26,23,23,21,21,20,2,1,14,15,15,12,12,7,3,0,2,3,0,2,5,0,4,0,4,0,0,4,0,0,4,4,4,29
Masters Women 1 (40-49),719,719,719,560,51,69,560,81,702,668,660,665,633,625,607,601,602,166,72,466,537,530,474,473,431,140,29,240,140,29,239,341,61,343,61,309,27,61,310,27,61,370,397,397,717
Masters Women 2 (50-59),287,287,287,220,15,37,220,26,278,261,259,258,239,237,227,228,224,51,23,171,191,187,169,168,150,60,12,69,60,12,69,117,20,116,20,100,12,20,101,12,20,121,133,133,285
Masters Women 3 (60-69),26,26,26,21,0,3,21,2,24,21,22,21,18,18,17,16,16,8,5,7,12,12,9,9,8,3,1,4,3,1,4,8,0,8,0,7,1,0,7,1,0,7,8,8,26
Senior Men (23-39),6042,6042,6042,4555,501,700,4555,674,5943,5737,5703,5706,5511,5515,5366,5315,5374,1325,639,4360,4955,4917,4701,4687,4354,1493,367,2247,1481,355,2248,3418,586,3423,586,3046,344,580,3044,344,580,3628,3972,3967,6036
Senior Women (23-39),646,646,646,519,31,69,518,51,630,614,606,607,585,579,561,558,563,156,70,440,498,493,460,457,421,134,33,237,134,33,234,326,66,324,66,284,32,66,285,32,66,348,380,380,645


In [80]:
df.loc[1613]

Year                                                        2017
name_mask                       4db319a671ab6e2b099df28df89f85c4
gender                                                      Male
category                                   Masters Men 1 (40-49)
Chamonix to Delevret                                      8.6826
Chamonix to La Charme                                        NaN
Chamonix to St-Gervais                                       NaN
Delevret to St-Gervais                                   8.11842
La Charme to St-Gervais                                      NaN
St-Gervais to Contamines                                 10.7799
Contamines to La Balme                                   14.7181
La Balme to Bonhomme                                     22.7212
Bonhomme to Chapieux                                     10.8431
Chapieux to Col Seigne                                   17.9725
Col Seigne to Lac Combal                                 10.3231
Lac Combal to Mt-Favre   

In [81]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Male,Senior Men (23-39),7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Male,Masters Men 1 (40-49),8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,2015,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,2009,00156070bc3ed3fd8b6b87dabd51c2fc,Male,Senior Men (23-39),NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


# Filter out runners who only ran one race

In [82]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [83]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
10507,2013,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
12954,2011,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.839080,NaN,NaN,2370.466667,1133.750000,13.945114
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387


# Merge

In [84]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [85]:
pb.head()

,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
10507,003cc0338235bb1862eb7b9bdd31ec9c,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
12954,006db776bb4048de7f357c632a8778ac,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
8431,00a7ade381ebb6d25e1aacb0bc7129e9,Male,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,2456.833333,1245.100000,15.121991
15237,00b39183bebaffe426e376f7efc8ae1b,Male,NaN,NaN,7.547287,NaN,NaN,9.362460,12.662551,16.833333,11.153595,13.598706,8.969388,17.229167,9.374074,10.439024,NaN,NaN,21.806122,10.963964,12.903846,19.774074,9.646667,11.486905,16.872054,NaN,NaN,15.801587,NaN,NaN,20.562893,NaN,13.461111,NaN,19.062771,NaN,NaN,14.969697,NaN,NaN,12.225225,2192.983333,899.683333,13.769848
6180,011b597a8c4121461b659f50dbd19d5a,Male,8.411765,NaN,NaN,7.717105,NaN,10.508091,12.713992,19.566667,11.323529,15.192557,23.520408,19.091667,11.370370,12.223577,29.333333,27.775,NaN,15.608108,16.535256,24.925926,11.051667,14.361905,NaN,NaN,21.212389,NaN,NaN,13.040816,25.194969,NaN,12.630556,NaN,21.954545,NaN,NaN,17.929293,NaN,NaN,12.461712,2595.100000,1325.850000,16.626208


In [86]:
npb.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387
3763,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.0,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
17546,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,29.197222,10.140777,13.462963,17.600000,11.225490,14.611650,11.180272,17.779167,10.611111,10.662602,NaN,NaN,22.261905,12.840090,13.230769,20.633333,10.486667,11.557143,17.387205,NaN,NaN,15.240741,NaN,NaN,20.226415,NaN,13.408333,NaN,19.309524,NaN,NaN,11.808081,NaN,NaN,9.322072,2255.450000,998.466667,14.964501


In [87]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [88]:
#merge = merge.groupby('name_mask').head(1)

In [89]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [90]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [91]:
comb

,category,gender
0,Senior Men (23-39),Male
1,Senior Men (23-39),Male
2,Senior Men (23-39),Male
3,Masters Men 1 (40-49),Male
4,Masters Men 1 (40-49),Male
5,Masters Men 1 (40-49),Male
6,Masters Men 1 (40-49),Male
7,Senior Men (23-39),Male
8,Senior Men (23-39),Male
9,Senior Men (23-39),Male


In [92]:
merge.loc[1613]

Year                                                           2015
name_mask                          fbac5ac0b66892696a502c9f3de3ee0e
gender                                                         Male
category                                          Youth Men (20-22)
Chamonix to Delevret                                        7.48897
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                      5.99342
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                    7.80583
Contamines to La Balme                                      9.45885
La Balme to Bonhomme                                        14.3242
Bonhomme to Chapieux                                        8.26471
Chapieux to Col Seigne                                      11.6942
Col Seigne to Lac Combal                        

In [93]:
merge.groupby('category').count()

,Year,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),690,690,690,484,65,96,484,88,687,688,689,690,689,688,687,674,689,144,77,612,687,686,689,690,690,289,122,279,289,122,279,632,57,633,57,510,122,57,510,122,57,568,690,690,690,690,509,56,113,509,60,685,689,686,689,688,690,687,677,690,106,59,631,685,682,689,690,690,308,92,290,308,92,288,610,77,613,77,521,92,76,521,92,77,598,690,690,690
Masters Men 2 (50-59),290,290,290,233,14,30,233,22,290,289,290,290,289,290,288,288,288,79,39,250,289,290,290,290,290,99,42,148,99,42,149,255,34,256,34,213,42,34,213,42,34,248,290,290,290,290,167,33,67,168,51,286,289,286,288,290,290,290,283,290,32,24,265,290,283,290,290,290,164,39,87,164,39,86,268,22,268,22,228,39,22,229,39,22,251,290,290,290
Masters Men 3 (60-69),31,31,31,26,5,0,25,4,31,31,31,31,31,31,31,31,31,7,4,26,31,31,31,31,31,8,6,17,8,6,17,26,5,26,5,20,6,5,20,6,5,25,31,31,31,31,12,0,15,12,4,31,31,30,31,31,31,31,31,31,0,0,31,31,31,31,31,31,23,6,2,23,6,2,30,1,30,1,24,6,1,24,6,1,25,31,31,31
Masters Men 4 (70-79),1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1
Masters Women 1 (40-49),46,46,46,35,3,6,35,4,46,46,46,46,46,46,45,46,46,16,9,36,46,46,46,46,46,16,3,27,16,3,27,43,3,43,3,40,3,3,40,3,3,43,46,46,46,46,38,1,7,38,1,46,46,46,46,46,46,46,46,46,10,3,43,46,46,46,46,46,13,6,27,13,6,27,36,10,36,10,30,6,10,30,6,10,40,46,46,46
Masters Women 2 (50-59),14,14,14,11,1,2,11,0,14,14,14,14,14,14,14,14,14,4,1,13,14,14,14,14,14,5,2,7,5,2,7,12,2,12,2,10,2,2,10,2,2,12,14,14,14,14,12,0,1,12,1,14,14,14,14,14,14,14,14,14,0,0,14,14,14,14,14,14,7,2,5,7,2,5,13,1,13,1,11,2,1,11,2,1,12,14,14,14
Masters Women 3 (60-69),2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,1,1,1,2,2,2,2,2,0,0,2,0,0,2,2,0,2,0,2,0,0,2,0,0,2,2,2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,0,0,2,2,2,2,2,2,2,0,0,2,0,0,2,0,2,0,2,0,0,2,0,0,2,2,2,2
Senior Men (23-39),501,501,501,318,59,89,319,84,497,499,500,500,499,499,

In [94]:
#merge = merge[merge.category != 'Youth Men (20-22)']
#merge = merge[merge.category != 'Masters Women 3 (60-69)']
#merge = merge[merge.category != 'Masters Men 4 (70-79)']

In [95]:
# Update pb and npb to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
npb = merge.loc[:, ~merge.columns.str.contains('pb')]

In [96]:
pb.head()

,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
3,Male,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,2456.833333,1245.100000,15.121991
4,Male,NaN,NaN,7.547287,NaN,NaN,9.362460,12.662551,16.833333,11.153595,13.598706,8.969388,17.229167,9.374074,10.439024,NaN,NaN,21.806122,10.963964,12.903846,19.774074,9.646667,11.486905,16.872054,NaN,NaN,15.801587,NaN,NaN,20.562893,NaN,13.461111,NaN,19.062771,NaN,NaN,14.969697,NaN,NaN,12.225225,2192.983333,899.683333,13.769848


In [97]:
npb.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387
3,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.0,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
4,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,29.197222,10.140777,13.462963,17.600000,11.225490,14.611650,11.180272,17.779167,10.611111,10.662602,NaN,NaN,22.261905,12.840090,13.230769,20.633333,10.486667,11.557143,17.387205,NaN,NaN,15.240741,NaN,NaN,20.226415,NaN,13.408333,NaN,19.309524,NaN,NaN,11.808081,NaN,NaN,9.322072,2255.450000,998.466667,14.964501


# Define Functions

In [98]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(0)

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
40,2015,07293d6206c13d09a85044be9e4963fc,Male,Senior Men (23-39),8.903186,NaN,NaN,7.578947,NaN,10.453074,12.619342,17.436364,9.307190,12.944984,19.156463,18.104167,7.655556,9.682927,34.074074,20.391667,NaN,11.891892,13.105769,24.907407,10.778333,13.861905,NaN,NaN,20.306785,NaN,NaN,9.568027,23.845912,NaN,8.997222,NaN,22.961039,NaN,NaN,11.126263,NaN,NaN,6.846847,2308.916667,1039.666667,14.660214
83,2013,0de585e8522f24147edec7692c655feb,Male,Senior Men (23-39),9.654412,NaN,NaN,7.048246,NaN,11.682848,12.440329,18.636364,8.405229,15.037217,8.785714,22.533333,9.344444,10.142276,NaN,NaN,29.319728,12.677928,12.820513,25.881481,9.910000,13.589286,16.690236,NaN,NaN,13.068783,NaN,NaN,25.823899,NaN,9.702778,NaN,21.945887,NaN,NaN,13.378788,NaN,NaN,9.993243,2362.733333,1127.783333,14.521373
105,2011,11c24a3c9ee9fcc173d9c21fa29c2bee,Male,Senior Men (23-39),8.547794,NaN,NaN,7.914474,NaN,8.734628,10.627572,16.215152,9.258170,13.211974,9.673469,15.450000,11.303704,10.609756,NaN,NaN,25.843537,11.731982,12.285256,21.618519,14.138333,15.263095,NaN,24.518248,NaN,NaN,20.773504,NaN,18.229560,NaN,11.741667,NaN,NaN,11.129630,NaN,NaN,11.429119,NaN,NaN,2392.733333,1156.016667,13.923876
106,2008,11c24a3c9ee9fcc173d9c21fa29c2bee,Male,Senior Men (23-39),NaN,4.958696,NaN,NaN,8.022222,8.572816,10.600823,15.254545,9.849673,12.626214,9.139456,15.066667,NaN,18.711382,NaN,NaN,20.064626,11.299550,11.544872,18.477778,11.096667,13.010714,18.638047,NaN,NaN,19.174603,NaN,NaN,25.094340,NaN,15.813889,NaN,24.173160,NaN,NaN,17.696970,NaN,NaN,12.558559,2271.200000,1014.216667,14.410707
118,2009,138b90e5dfce79bb890a7c8393d22982,Male,Senior Men (23-39),NaN,NaN,9.158915,NaN,NaN,10.208738,12.047325,18.227273,11.359477,14.207120,8.238095,17.487500,8.911111,12.333333,NaN,NaN,23.272109,10.795045,11.310897,19.744444,11.431667,11.636905,16.858586,NaN,NaN,13.838624,NaN,NaN,20.877358,NaN,12.711111,NaN,22.694805,NaN,NaN,12.479798,NaN,NaN,8.990991,2247.433333,954.133333,13.861793
138,2016,16f90397fb6a11351cb330e03f8edc1b,Male,Senior Men (23-39),8.291667,NaN,NaN,6.407895,NaN,8.988673,10.921811,15.424242,8.460784,12.478964,18.404762,16.808333,9.644444,11.483740,20.796296,NaN,16.095238,13.918919,13.474359,19.929630,10.646667,12.414286,NaN,NaN,15.141593,NaN,NaN,12.843537,22.644654,NaN,13.088889,NaN,17.588745,NaN,NaN,15.888889,NaN,NaN,10.518018,2185.816667,865.783333,13.692201
150,2009,17d416171d9f6e3dab9a4a2561dd4575,Male,Senior Men (23-39),NaN,NaN,7.855814,NaN,NaN,8.398058,10.109053,14.390909,8.196078,12.108414,10.081633,21.837500,9.574074,10.048780,NaN,NaN,20.860544,16.756757,12.028846,20.237037,8.771667,11.932143,15.611111,NaN,NaN,14.126984,NaN,NaN,21.496855,NaN,11.733333,NaN,21.467532,NaN,N

In [99]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    #euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    euclidean_distance = euclidean_distance.head(10)
    euclidean_distance_1 = euclidean_distance.head(1)
    euclidean_distance_2 = euclidean_distance.head(2)
    euclidean_distance_5 = euclidean_distance.head(5)
    euclidean_distance_20 = euclidean_distance.head(20)
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(0)

1439    11.253756
664     16.170319
415     30.498519
1127    46.203162
1087    50.055875
276     53.417078
951     61.340387
83      66.482538
1284    70.494002
741     74.141372
Name: 0, dtype: float64

In [100]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0).head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
1439,2014,de99417dced7762c0b3cebbd7e98055f,Male,Senior Men (23-39),7.985294,NaN,NaN,6.383772,NaN,10.182848,13.549383,15.912121,9.202614,15.025890,9.469388,21.008333,8.385185,11.955285,NaN,NaN,26.289116,11.592342,11.839744,23.585185,9.601667,15.147619,NaN,NaN,17.762537,NaN,NaN,14.472789,24.345912,NaN,12.725000,NaN,21.937229,NaN,NaN,14.883838,NaN,NaN,7.371622,2318.133333,1106.400000,14.192280,Male,7.661765,NaN,NaN,6.030702,NaN,9.414239,12.446502,14.700000,8.771242,12.987055,19.044218,21.841667,8.459259,9.768293,47.888889,NaN,15.438776,10.894144,11.660256,24.492593,8.551667,12.029762,NaN,NaN,18.497050,NaN,NaN,11.914966,19.415094,NaN,10.997222,NaN,18.071429,NaN,NaN,15.969697,NaN,NaN,9.500000,2188.550000,868.516667,14.657859
664,2014,67870601366b233f04bdab9d3eda22d1,Male,Senior Men (23-39),7.601716,NaN,NaN,6.221491,NaN,9.451456,13.199588,16.615152,8.620915,13.956311,9.176871,19.737500,11.248148,10.658537,NaN,NaN,22.479592,11.623874,14.586538,21.288889,11.016667,14.070238,NaN,NaN,17.457227,NaN,NaN,12.316327,23.462264,NaN,10.752778,NaN,26.049784,NaN,NaN,19.767677,NaN,NaN,11.119369,2321.816667,1110.083333,14.269954,Male,NaN,NaN,NaN,NaN,28.202778,8.752427,10.965021,15.639394,8.578431,12.710356,9.020408,16.545833,NaN,18.422764,NaN,NaN,18.843537,12.038288,10.910256,21.225926,11.085000,11.665476,16.653199,NaN,NaN,13.740741,NaN,NaN,25.688679,NaN,14.894444,NaN,24.712121,NaN,NaN,17.186869,NaN,NaN,10.042793,2200.883333,943.900000,15.342034
415,2014,40607767095ee0cb77a09a3c21e849a7,Male,Senior Men (23-39),9.354167,NaN,NaN,8.320175,NaN,10.438511,11.810700,18.436364,10.767974,15.687702,10.462585,22.250000,9.525926,10.817073,NaN,NaN,24.972789,11.885135,12.576923,20.633333,10.536667,13.141667,NaN,NaN,17.417404,NaN,NaN,12.812925,23.427673,NaN,10.608333,NaN,22.075758,NaN,NaN,15.651515,NaN,NaN,9.099099,2333.316667,1121.583333,14.279600,Male,8.708333,NaN,NaN,6.807018,NaN,9.344660,11.956790,16.675758,9.637255,15.304207,20.908163,17.450000,8.040741,9.678862,37.462963,21.466667,NaN,12.869369,12.778846,19.900000,8.688333,12.89047

In [101]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_mean_pb(neighbours)

14.027846259117817

In [102]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
neighbours = neighbours.filter(like='pb')
get_mean_pace(neighbours)

Chamonix to Delevret_pb               8.038807
Chamonix to La Charme_pb                   NaN
Chamonix to St-Gervais_pb             8.062532
Delevret to St-Gervais_pb             6.482091
La Charme to St-Gervais_pb           28.202778
St-Gervais to Contamines_pb           9.206149
Contamines to La Balme_pb            11.577778
La Balme to Bonhomme_pb              16.703333
Bonhomme to Chapieux_pb               9.254248
Chapieux to Col Seigne_pb            13.874757
Col Seigne to Lac Combal_pb          12.093878
Lac Combal to Mt-Favre_pb            19.061667
Mt-Favre to Checruit_pb               8.749794
Checruit to Courmayeur_pb            10.647154
Courmayeur to Courmayeur2_pb         37.469136
Courmayeur2 to Bertone_pb            21.466667
Courmayeur to Bertone_pb             22.144369
Bertone to Bonatti_pb                12.112838
Bonatti to Arnouvaz_pb               11.314423
Arnouvaz to Col Ferret_pb            21.764444
Col Ferret to La Fouly_pb             9.173500
La Fouly to C

In [103]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_npb(neighbours)

13.815978482511847

In [104]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_pb(neighbours)

11.830188232425401

In [105]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


8.062532299741603

In [106]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

12.144611429805657

# Calculate Predictions

In [107]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [108]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace']]
df

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace
0,14.183180,13.489501,12.144611,14.027846
1,15.188612,13.489501,11.791693,14.582734
2,16.367387,13.945114,14.739157,16.261925
3,17.602140,15.121991,13.948090,15.921597
4,14.964501,13.769848,10.696283,12.627868
5,16.136556,16.626208,13.617350,15.373875
6,14.533689,13.438194,12.637283,13.269206
7,16.813847,18.265522,15.074019,16.031507
8,15.182644,11.779689,13.497942,14.863906
9,16.945035,14.825623,15.008913,15.933368


In [109]:
df = comb.join(df)

In [110]:
df.to_csv('../../data/pace_predictions.csv')

In [111]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


8.062532299741603

In [112]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

12.144611429805657

# Mean Pace Recommendations

In [113]:
def update_pace(line):
    index = int(line['Chamonix to Delevret_pb'])
    neighbours = find_nearest_neighbours(index)
    line['Chamonix to St-Gervais_pb'] = Cham_to_StG(neighbours)
    line['Chamonix to La Charme_pb'] = Cham_to_LaCh(neighbours)
    line['Delevret to St-Gervais_pb'] = Del_to_StG(neighbours)
    line['La Charme to St-Gervais_pb'] = LaCh_to_StG(neighbours)
    line['St-Gervais to Contamines_pb'] = StG_to_Cont(neighbours)
    line['Contamines to La Balme_pb'] = Cont_to_LaB(neighbours)
    line['La Balme to Bonhomme_pb'] = LaB_to_Bon(neighbours)
    line['Bonhomme to Chapieux_pb'] = Bon_to_Chap(neighbours)
    line['Chapieux to Col Seigne_pb'] = Chap_to_ColS(neighbours)
    line['Col Seigne to Lac Combal_pb'] = ColS_to_Lac(neighbours)
    line['Lac Combal to Mt-Favre_pb'] = Lac_to_MtF(neighbours)
    line['Mt-Favre to Checruit_pb '] = MtF_to_Chec(neighbours)
    line['Checruit to Courmayeur_pb'] = Chec_to_Courm(neighbours)
    line['Courmayeur to Courmayeur2_pb'] = Courm_to_Courm2(neighbours)
    line['Courmayeur2 to Bertone_pb'] = Courm2_to_Bertone(neighbours)
    line['Courmayeur to Bertone_pb'] = Courm_to_Bertone(neighbours)
    line['Bertone to Bonatti_pb'] = Bertone_to_Bon(neighbours)
    line['Bonatti to Arnouvaz_pb'] = Bon_to_Arn(neighbours)
    line['Arnouvaz to Col Ferret_pb'] = Arn_to_Col(neighbours)
    line['Col Ferret to La Fouly_pb'] = Col_to_LaF(neighbours)
    line['La Fouly to Champex La_pb'] = LaF_to_Champ(neighbours)
    line['Champex La to Bovine_pb'] = Champ_to_Bov(neighbours)
    line['Champex La to Mortigny_pb'] = Champ_to_Mort(neighbours)
    line['Champex La to Giete_pb'] = Champ_to_Giete(neighbours)
    line['Bovine to Trient_pb'] = Bov_to_Trient(neighbours)
    line['Mortigny to Trient_pb'] = Mort_to_Trient(neighbours)
    line['Giete to Trient_pb'] = Giete_to_Trient(neighbours)
    line['Trient to La Catogne_pb'] = Trient_to_LaCa(neighbours)
    line['Trient to Les Tseppe_pb'] = Trient_to_LesSt(neighbours)
    line['La Catogne to Vallorcine_pb'] = LaCa_to_Vall(neighbours)
    line['Les Tseppe to Vallorcine_pb'] = LesSt_to_Vall(neighbours)
    line['Vallorcine to Tete aux Vents_pb'] = Vall_to_tAV(neighbours)
    line['Vallorcine to Argentiere_pb'] = Vall_to_Arg(neighbours)
    line['Vallorcine to Col Montet_pb'] = Vall_to_Col(neighbours)
    line['Tete aux Vents to Flegere_pb'] = tAV_to_Fleg(neighbours)
    line['Argentiere to Chamonix_pb'] = Arg_to_Cham(neighbours)
    line['Col Montet to Flegere_pb'] = Col_to_Fleg(neighbours)
    line['Flegere to Chamonix_pb'] = Fleg_to_Cham(neighbours)
    
    line['Chamonix to Delevret_pb'] = Cham_to_Del(neighbours)
    
    return line

In [114]:
num_of_runs1 = merge.time.count()
filler_data = np.arange(0.0, num_of_runs1)
df1 = pd.DataFrame(
    { 
     'Chamonix to Delevret_pb':filler_data,            
'Chamonix to La Charme_pb':filler_data,            
     'Chamonix to St-Gervais_pb':filler_data,      
'Delevret to St-Gervais_pb':filler_data,           
'La Charme to St-Gervais_pb':filler_data,          
'St-Gervais to Contamines_pb':filler_data,         
'Contamines to La Balme_pb':filler_data,           
'La Balme to Bonhomme_pb':filler_data,             
     'Bonhomme to Chapieux_pb':filler_data,              
'Chapieux to Col Seigne_pb':filler_data,           
'Col Seigne to Lac Combal_pb':filler_data,         
'Lac Combal to Mt-Favre_pb':filler_data,           
'Mt-Favre to Checruit_pb':filler_data,             
'Checruit to Courmayeur_pb':filler_data,           
'Courmayeur to Courmayeur2_pb':filler_data,        
'Courmayeur2 to Bertone_pb':filler_data,           
'Courmayeur to Bertone_pb':filler_data,           
'Bertone to Bonatti_pb':filler_data,               
'Bonatti to Arnouvaz_pb':filler_data,              
'Arnouvaz to Col Ferret_pb':filler_data,           
'Col Ferret to La Fouly_pb':filler_data,           
'La Fouly to Champex La_pb':filler_data,           
'Champex La to Bovine_pb':filler_data,             
'Champex La to Mortigny_pb':filler_data,           
'Champex La to Giete_pb':filler_data,             
'Bovine to Trient_pb':filler_data,                 
'Mortigny to Trient_pb':filler_data,               
'Giete to Trient_pb':filler_data,                  
'Trient to La Catogne_pb':filler_data,             
'Trient to Les Tseppe_pb':filler_data,             
'La Catogne to Vallorcine_pb':filler_data,         
'Les Tseppe to Vallorcine_pb':filler_data,         
'Vallorcine to Tete aux Vents_pb':filler_data,     
'Vallorcine to Argentiere_pb':filler_data,         
'Vallorcine to Col Montet_pb':filler_data,         
'Tete aux Vents to Flegere_pb':filler_data,        
'Argentiere to Chamonix_pb':filler_data,           
'Col Montet to Flegere_pb':filler_data,            
'Flegere to Chamonix_pb':filler_data             
    }
)

df1 = df1.apply(lambda line: update_pace(line), axis=1)
df1 = df1[['Chamonix to Delevret_pb',            
'Chamonix to La Charme_pb',            
     'Chamonix to St-Gervais_pb',      
'Delevret to St-Gervais_pb',           
'La Charme to St-Gervais_pb',          
'St-Gervais to Contamines_pb',         
'Contamines to La Balme_pb',           
'La Balme to Bonhomme_pb',             
     'Bonhomme to Chapieux_pb',              
'Chapieux to Col Seigne_pb',           
'Col Seigne to Lac Combal_pb',         
'Lac Combal to Mt-Favre_pb',           
'Mt-Favre to Checruit_pb',             
'Checruit to Courmayeur_pb',           
'Courmayeur to Courmayeur2_pb',        
'Courmayeur2 to Bertone_pb',           
'Courmayeur to Bertone_pb',           
'Bertone to Bonatti_pb',               
'Bonatti to Arnouvaz_pb',              
'Arnouvaz to Col Ferret_pb',           
'Col Ferret to La Fouly_pb',           
'La Fouly to Champex La_pb',           
'Champex La to Bovine_pb',             
'Champex La to Mortigny_pb',           
'Champex La to Giete_pb',             
'Bovine to Trient_pb',                 
'Mortigny to Trient_pb',               
'Giete to Trient_pb',                  
'Trient to La Catogne_pb',             
'Trient to Les Tseppe_pb',             
'La Catogne to Vallorcine_pb',         
'Les Tseppe to Vallorcine_pb',         
'Vallorcine to Tete aux Vents_pb',     
'Vallorcine to Argentiere_pb',         
'Vallorcine to Col Montet_pb',         
'Tete aux Vents to Flegere_pb',        
'Argentiere to Chamonix_pb',           
'Col Montet to Flegere_pb',            
'Flegere to Chamonix_pb']]
df1.head()

,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,8.038807,NaN,8.062532,6.482091,28.202778,9.206149,11.577778,16.703333,9.254248,13.874757,12.093878,19.061667,0.0,10.647154,37.469136,21.466667,22.144369,12.112838,11.314423,21.764444,9.173500,11.893333,17.158586,NaN,17.712684,13.653439,NaN,12.892517,22.349406,21.319444,11.170679,16.905172,20.405002,NaN,21.382883,14.189675,NaN,21.088095,9.974550
1,8.466708,5.332609,8.601034,7.013523,9.122222,9.998544,12.169547,17.256061,9.542484,14.614887,10.113946,18.645833,1.0,11.119919,52.129630,23.975000,26.674981,12.744595,13.478526,23.812963,10.057500,12.808571,18.344396,NaN,19.264012,14.954271,NaN,12.071429,23.867575,20.614583,12.546296,17.557471,22.797258,NaN,22.085586,13.929854,NaN,19.719048,9.894369
2,8.829112,NaN,7.797674,7.421053,NaN,10.463269,13.812346,18.369697,10.182353,15.117314,14.964286,18.820000,2.0,11.726829,60.400000,24.412500,23.923226,12.870946,13.841346,23.382222,10.715500,13.719643,19.510943,NaN,20.360804,19.014550,NaN,14.106718,27.861635,25.041667,13.701235,17.640805,22.518278,NaN,24.819820,17.861953,NaN,19.440476,11.978829
3,9.387255,4.891304,NaN,8.144981,8.202778,10.769417,13.256584,18.343030,10.970261,15.052427,11.793537,19.380833,3.0,12.865041,42.009259,23.102083,27.789966,13.392568,13.109776,27.298519,11.673000,14.514286,21.796857,17.327251,22.652163,16.795414,16.446581,15.661565,26.940601,32.770833,13.861728,19.758621,22.816558,10.492063,24.626126,16.268939,11.011494,21.209524,11.693694
4,8.028186,3.999275,6.944574,6.708647,6.425000,8.507282,11.073251,15.313030,8.698366,13.622816,7.558163,16.120417,4.0,9.584146,NaN,NaN,22.703741,10.156982,10.311859,19.453704,9.584333,11.186548,15.312710,13.709854,14.272124,12.794974,18.397970,9.790816,21.071279,16.107639,11.616667,12.370690,22.342424,11.721561,13.900901,16.148485,12.182471,15.909524,12.789790


# Best Pace Recommendations

In [115]:
def Cham_2_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].min()

def Cham_2_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].min()

def Cham_2_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].min()

def Del_2_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].min()

def LaCh_2_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].min()

def StG_2_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].min()

def Cont_2_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].min()

def LaB_2_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].min()

def Bon_2_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].min()

def Chap_2_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].min()

def ColS_2_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].min()

def Lac_2_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].min()

def MtF_2_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].min()

def Chec_2_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].min()

def Courm_2_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].min()

def Courm2_2_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].min()

def Courm_2_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].min()

def Bertone_2_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].min()

def Bon_2_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].min()

def Arn_2_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].min()

def Col_2_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].min()

def LaF_2_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].min()

def Champ_2_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].min()

def Champ_2_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].min()

def Champ_2_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].min()

def Bov_2_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].min()

def Mort_2_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].min()

def Giete_2_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].min()

def Trient_2_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].min()

def Trient_2_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].min()

def LaCa_2_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].min()

def LesSt_2_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].min()

def Vall_2_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].min()

def Vall_2_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].min()

def Vall_2_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].min()

def tAV_2_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].min()

def Arg_2_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].min()

def Col_2_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].min()

def Fleg_2_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].min()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_2_StG(neighbours)
#Cham_2_LaCh(neighbours)
#Cham_2_StG(neighbours)   


7.0395348837209299

In [116]:
def update_best(sta):
    index = int(sta['Chamonix to Delevret_pb'])
    neighbours = find_nearest_neighbours(index)
    sta['Chamonix to St-Gervais_pb'] = Cham_2_StG(neighbours)
    sta['Chamonix to La Charme_pb'] = Cham_2_LaCh(neighbours)
    sta['Delevret to St-Gervais_pb'] = Del_2_StG(neighbours)
    sta['La Charme to St-Gervais_pb'] = LaCh_2_StG(neighbours)
    sta['St-Gervais to Contamines_pb'] = StG_2_Cont(neighbours)
    sta['Contamines to La Balme_pb'] = Cont_2_LaB(neighbours)
    sta['La Balme to Bonhomme_pb'] = LaB_2_Bon(neighbours)
    sta['Bonhomme to Chapieux_pb'] = Bon_2_Chap(neighbours)
    sta['Chapieux to Col Seigne_pb'] = Chap_2_ColS(neighbours)
    sta['Col Seigne to Lac Combal_pb'] = ColS_2_Lac(neighbours)
    sta['Lac Combal to Mt-Favre_pb'] = Lac_2_MtF(neighbours)
    sta['Mt-Favre to Checruit_pb '] = MtF_2_Chec(neighbours)
    sta['Checruit to Courmayeur_pb'] = Chec_2_Courm(neighbours)
    sta['Courmayeur to Courmayeur2_pb'] = Courm_2_Courm2(neighbours)
    sta['Courmayeur2 to Bertone_pb'] = Courm2_2_Bertone(neighbours)
    sta['Courmayeur to Bertone_pb'] = Courm_2_Bertone(neighbours)
    sta['Bertone to Bonatti_pb'] = Bertone_2_Bon(neighbours)
    sta['Bonatti to Arnouvaz_pb'] = Bon_2_Arn(neighbours)
    sta['Arnouvaz to Col Ferret_pb'] = Arn_2_Col(neighbours)
    sta['Col Ferret to La Fouly_pb'] = Col_2_LaF(neighbours)
    sta['La Fouly to Champex La_pb'] = LaF_2_Champ(neighbours)
    sta['Champex La to Bovine_pb'] = Champ_2_Bov(neighbours)
    sta['Champex La to Mortigny_pb'] = Champ_2_Mort(neighbours)
    sta['Champex La to Giete_pb'] = Champ_2_Giete(neighbours)
    sta['Bovine to Trient_pb'] = Bov_2_Trient(neighbours)
    sta['Mortigny to Trient_pb'] = Mort_2_Trient(neighbours)
    sta['Giete to Trient_pb'] = Giete_2_Trient(neighbours)
    sta['Trient to La Catogne_pb'] = Trient_2_LaCa(neighbours)
    sta['Trient to Les Tseppe_pb'] = Trient_2_LesSt(neighbours)
    sta['La Catogne to Vallorcine_pb'] = LaCa_2_Vall(neighbours)
    sta['Les Tseppe to Vallorcine_pb'] = LesSt_2_Vall(neighbours)
    sta['Vallorcine to Tete aux Vents_pb'] = Vall_2_tAV(neighbours)
    sta['Vallorcine to Argentiere_pb'] = Vall_2_Arg(neighbours)
    sta['Vallorcine to Col Montet_pb'] = Vall_2_Col(neighbours)
    sta['Tete aux Vents to Flegere_pb'] = tAV_2_Fleg(neighbours)
    sta['Argentiere to Chamonix_pb'] = Arg_2_Cham(neighbours)
    sta['Col Montet to Flegere_pb'] = Col_2_Fleg(neighbours)
    sta['Flegere to Chamonix_pb'] = Fleg_2_Cham(neighbours)
    
    sta['Chamonix to Delevret_pb'] = Cham_2_Del(neighbours)
    
    return sta

In [117]:
num_of_runs2 = merge.time.count()
filler_data = np.arange(0.0, num_of_runs2)
df2 = pd.DataFrame(
    { 
     'Chamonix to Delevret_pb':filler_data,            
'Chamonix to La Charme_pb':filler_data,            
     'Chamonix to St-Gervais_pb':filler_data,      
'Delevret to St-Gervais_pb':filler_data,           
'La Charme to St-Gervais_pb':filler_data,          
'St-Gervais to Contamines_pb':filler_data,         
'Contamines to La Balme_pb':filler_data,           
'La Balme to Bonhomme_pb':filler_data,             
     'Bonhomme to Chapieux_pb':filler_data,              
'Chapieux to Col Seigne_pb':filler_data,           
'Col Seigne to Lac Combal_pb':filler_data,         
'Lac Combal to Mt-Favre_pb':filler_data,           
'Mt-Favre to Checruit_pb':filler_data,             
'Checruit to Courmayeur_pb':filler_data,           
'Courmayeur to Courmayeur2_pb':filler_data,        
'Courmayeur2 to Bertone_pb':filler_data,           
'Courmayeur to Bertone_pb':filler_data,           
'Bertone to Bonatti_pb':filler_data,               
'Bonatti to Arnouvaz_pb':filler_data,              
'Arnouvaz to Col Ferret_pb':filler_data,           
'Col Ferret to La Fouly_pb':filler_data,           
'La Fouly to Champex La_pb':filler_data,           
'Champex La to Bovine_pb':filler_data,             
'Champex La to Mortigny_pb':filler_data,           
'Champex La to Giete_pb':filler_data,             
'Bovine to Trient_pb':filler_data,                 
'Mortigny to Trient_pb':filler_data,               
'Giete to Trient_pb':filler_data,                  
'Trient to La Catogne_pb':filler_data,             
'Trient to Les Tseppe_pb':filler_data,             
'La Catogne to Vallorcine_pb':filler_data,         
'Les Tseppe to Vallorcine_pb':filler_data,         
'Vallorcine to Tete aux Vents_pb':filler_data,     
'Vallorcine to Argentiere_pb':filler_data,         
'Vallorcine to Col Montet_pb':filler_data,         
'Tete aux Vents to Flegere_pb':filler_data,        
'Argentiere to Chamonix_pb':filler_data,           
'Col Montet to Flegere_pb':filler_data,            
'Flegere to Chamonix_pb':filler_data             
    }
)

df2 = df2.apply(lambda sta: update_best(sta), axis=1)
df2 = df2[['Chamonix to Delevret_pb',            
'Chamonix to La Charme_pb',            
     'Chamonix to St-Gervais_pb',      
'Delevret to St-Gervais_pb',           
'La Charme to St-Gervais_pb',          
'St-Gervais to Contamines_pb',         
'Contamines to La Balme_pb',           
'La Balme to Bonhomme_pb',             
     'Bonhomme to Chapieux_pb',              
'Chapieux to Col Seigne_pb',           
'Col Seigne to Lac Combal_pb',         
'Lac Combal to Mt-Favre_pb',           
'Mt-Favre to Checruit_pb',             
'Checruit to Courmayeur_pb',           
'Courmayeur to Courmayeur2_pb',        
'Courmayeur2 to Bertone_pb',           
'Courmayeur to Bertone_pb',           
'Bertone to Bonatti_pb',               
'Bonatti to Arnouvaz_pb',              
'Arnouvaz to Col Ferret_pb',           
'Col Ferret to La Fouly_pb',           
'La Fouly to Champex La_pb',           
'Champex La to Bovine_pb',             
'Champex La to Mortigny_pb',           
'Champex La to Giete_pb',             
'Bovine to Trient_pb',                 
'Mortigny to Trient_pb',               
'Giete to Trient_pb',                  
'Trient to La Catogne_pb',             
'Trient to Les Tseppe_pb',             
'La Catogne to Vallorcine_pb',         
'Les Tseppe to Vallorcine_pb',         
'Vallorcine to Tete aux Vents_pb',     
'Vallorcine to Argentiere_pb',         
'Vallorcine to Col Montet_pb',         
'Tete aux Vents to Flegere_pb',        
'Argentiere to Chamonix_pb',           
'Col Montet to Flegere_pb',            
'Flegere to Chamonix_pb']]
df2.head()

,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,7.083333,NaN,7.039535,5.719298,28.202778,7.616505,9.611111,14.336364,7.807190,11.377023,7.047619,15.533333,0.0,8.211382,27.055556,21.466667,15.438776,10.894144,7.384615,16.029630,6.733333,10.545238,13.259259,NaN,15.528024,10.698413,NaN,8.710884,19.210692,21.319444,9.175000,16.905172,18.071429,NaN,21.382883,10.196970,NaN,21.088095,7.045045
1,7.136029,5.332609,8.237209,6.234649,9.122222,8.406149,9.057613,15.342424,4.624183,12.006472,7.489796,13.537500,1.0,9.231707,52.129630,23.975000,22.833333,10.310811,10.564103,16.733333,8.940000,10.070238,12.712121,NaN,17.942478,11.817460,NaN,10.557823,18.462264,20.614583,8.583333,17.557471,17.110390,NaN,22.085586,8.196970,NaN,19.719048,6.572072
2,7.780637,NaN,7.797674,5.971491,NaN,8.446602,10.261317,14.539394,7.526144,12.305825,8.278912,14.991667,2.0,9.239837,49.111111,23.975000,14.489796,10.770270,11.455128,20.507407,8.591667,11.072619,18.961279,NaN,14.879056,17.357143,NaN,10.557823,22.166667,25.041667,11.055556,17.640805,19.017316,NaN,24.819820,13.242424,NaN,19.440476,7.234234
3,8.175245,4.891304,NaN,6.304825,8.202778,8.524272,10.541152,16.118182,9.218954,13.134304,7.401361,17.537500,3.0,11.012195,36.092593,23.062500,24.656463,10.558559,10.086538,21.588889,9.701667,12.401190,17.274411,17.327251,18.904130,15.195767,16.446581,13.343537,20.380503,32.770833,11.522222,19.758621,20.359307,10.492063,24.626126,11.429293,11.011494,21.209524,8.355856
4,7.301471,3.999275,6.316279,5.710526,6.425000,6.966019,8.713992,13.027273,7.506536,10.956311,5.833333,12.170833,4.0,7.800813,NaN,NaN,18.598639,8.445946,8.791667,15.362963,7.600000,8.567857,13.008418,10.444039,13.889381,10.761905,15.320513,9.442177,16.452830,16.107639,9.622222,12.370690,19.887446,9.973545,13.900901,14.045455,9.500000,15.909524,8.418919


# K = 5 

## Define Functions

In [118]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(0)

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
40,2015,07293d6206c13d09a85044be9e4963fc,Male,Senior Men (23-39),8.903186,NaN,NaN,7.578947,NaN,10.453074,12.619342,17.436364,9.307190,12.944984,19.156463,18.104167,7.655556,9.682927,34.074074,20.391667,NaN,11.891892,13.105769,24.907407,10.778333,13.861905,NaN,NaN,20.306785,NaN,NaN,9.568027,23.845912,NaN,8.997222,NaN,22.961039,NaN,NaN,11.126263,NaN,NaN,6.846847,2308.916667,1039.666667,14.660214
83,2013,0de585e8522f24147edec7692c655feb,Male,Senior Men (23-39),9.654412,NaN,NaN,7.048246,NaN,11.682848,12.440329,18.636364,8.405229,15.037217,8.785714,22.533333,9.344444,10.142276,NaN,NaN,29.319728,12.677928,12.820513,25.881481,9.910000,13.589286,16.690236,NaN,NaN,13.068783,NaN,NaN,25.823899,NaN,9.702778,NaN,21.945887,NaN,NaN,13.378788,NaN,NaN,9.993243,2362.733333,1127.783333,14.521373
105,2011,11c24a3c9ee9fcc173d9c21fa29c2bee,Male,Senior Men (23-39),8.547794,NaN,NaN,7.914474,NaN,8.734628,10.627572,16.215152,9.258170,13.211974,9.673469,15.450000,11.303704,10.609756,NaN,NaN,25.843537,11.731982,12.285256,21.618519,14.138333,15.263095,NaN,24.518248,NaN,NaN,20.773504,NaN,18.229560,NaN,11.741667,NaN,NaN,11.129630,NaN,NaN,11.429119,NaN,NaN,2392.733333,1156.016667,13.923876
106,2008,11c24a3c9ee9fcc173d9c21fa29c2bee,Male,Senior Men (23-39),NaN,4.958696,NaN,NaN,8.022222,8.572816,10.600823,15.254545,9.849673,12.626214,9.139456,15.066667,NaN,18.711382,NaN,NaN,20.064626,11.299550,11.544872,18.477778,11.096667,13.010714,18.638047,NaN,NaN,19.174603,NaN,NaN,25.094340,NaN,15.813889,NaN,24.173160,NaN,NaN,17.696970,NaN,NaN,12.558559,2271.200000,1014.216667,14.410707
118,2009,138b90e5dfce79bb890a7c8393d22982,Male,Senior Men (23-39),NaN,NaN,9.158915,NaN,NaN,10.208738,12.047325,18.227273,11.359477,14.207120,8.238095,17.487500,8.911111,12.333333,NaN,NaN,23.272109,10.795045,11.310897,19.744444,11.431667,11.636905,16.858586,NaN,NaN,13.838624,NaN,NaN,20.877358,NaN,12.711111,NaN,22.694805,NaN,NaN,12.479798,NaN,NaN,8.990991,2247.433333,954.133333,13.861793
138,2016,16f90397fb6a11351cb330e03f8edc1b,Male,Senior Men (23-39),8.291667,NaN,NaN,6.407895,NaN,8.988673,10.921811,15.424242,8.460784,12.478964,18.404762,16.808333,9.644444,11.483740,20.796296,NaN,16.095238,13.918919,13.474359,19.929630,10.646667,12.414286,NaN,NaN,15.141593,NaN,NaN,12.843537,22.644654,NaN,13.088889,NaN,17.588745,NaN,NaN,15.888889,NaN,NaN,10.518018,2185.816667,865.783333,13.692201
150,2009,17d416171d9f6e3dab9a4a2561dd4575,Male,Senior Men (23-39),NaN,NaN,7.855814,NaN,NaN,8.398058,10.109053,14.390909,8.196078,12.108414,10.081633,21.837500,9.574074,10.048780,NaN,NaN,20.860544,16.756757,12.028846,20.237037,8.771667,11.932143,15.611111,NaN,NaN,14.126984,NaN,NaN,21.496855,NaN,11.733333,NaN,21.467532,NaN,N

In [119]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    #euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    #euclidean_distance = euclidean_distance.head(10)
   # euclidean_distance_1 = euclidean_distance.head(1)
    #euclidean_distance_2 = euclidean_distance.head(2)
    euclidean_distance_5 = euclidean_distance.head(5)
    #euclidean_distance_20 = euclidean_distance.head(20)
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(0)

1439     11.253756
664      16.170319
415      30.498519
1127     46.203162
1087     50.055875
276      53.417078
951      61.340387
83       66.482538
1284     70.494002
741      74.141372
665      75.397558
1336     77.681757
592      78.724871
40       78.811073
1458     79.467827
804      82.316050
672      86.193188
1036     87.035716
1050     87.281867
463      87.944371
152      94.632593
955      98.053782
177      99.026358
1060    100.612989
190     100.660517
106     104.855065
105     111.134709
537     112.555974
707     117.201509
1275    118.610154
769     119.282932
616     125.603123
979     132.449373
460     141.721499
118     164.646742
1508    165.978477
160     177.400300
1560    180.007351
1552    184.610146
615     185.937895
1053    189.519516
360     189.959389
1526    197.642647
1106    201.019217
1201    208.934575
653     209.866702
980     213.524088
1589    215.150893
1536    216.062354
982     218.705507
1528    226.085016
1163    233.130264
150     246.

In [120]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0).head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
1439,2014,de99417dced7762c0b3cebbd7e98055f,Male,Senior Men (23-39),7.985294,NaN,NaN,6.383772,NaN,10.182848,13.549383,15.912121,9.202614,15.025890,9.469388,21.008333,8.385185,11.955285,NaN,NaN,26.289116,11.592342,11.839744,23.585185,9.601667,15.147619,NaN,NaN,17.762537,NaN,NaN,14.472789,24.345912,NaN,12.725000,NaN,21.937229,NaN,NaN,14.883838,NaN,NaN,7.371622,2318.133333,1106.400000,14.192280,Male,7.661765,NaN,NaN,6.030702,NaN,9.414239,12.446502,14.700000,8.771242,12.987055,19.044218,21.841667,8.459259,9.768293,47.888889,NaN,15.438776,10.894144,11.660256,24.492593,8.551667,12.029762,NaN,NaN,18.497050,NaN,NaN,11.914966,19.415094,NaN,10.997222,NaN,18.071429,NaN,NaN,15.969697,NaN,NaN,9.500000,2188.550000,868.516667,14.657859
664,2014,67870601366b233f04bdab9d3eda22d1,Male,Senior Men (23-39),7.601716,NaN,NaN,6.221491,NaN,9.451456,13.199588,16.615152,8.620915,13.956311,9.176871,19.737500,11.248148,10.658537,NaN,NaN,22.479592,11.623874,14.586538,21.288889,11.016667,14.070238,NaN,NaN,17.457227,NaN,NaN,12.316327,23.462264,NaN,10.752778,NaN,26.049784,NaN,NaN,19.767677,NaN,NaN,11.119369,2321.816667,1110.083333,14.269954,Male,NaN,NaN,NaN,NaN,28.202778,8.752427,10.965021,15.639394,8.578431,12.710356,9.020408,16.545833,NaN,18.422764,NaN,NaN,18.843537,12.038288,10.910256,21.225926,11.085000,11.665476,16.653199,NaN,NaN,13.740741,NaN,NaN,25.688679,NaN,14.894444,NaN,24.712121,NaN,NaN,17.186869,NaN,NaN,10.042793,2200.883333,943.900000,15.342034
415,2014,40607767095ee0cb77a09a3c21e849a7,Male,Senior Men (23-39),9.354167,NaN,NaN,8.320175,NaN,10.438511,11.810700,18.436364,10.767974,15.687702,10.462585,22.250000,9.525926,10.817073,NaN,NaN,24.972789,11.885135,12.576923,20.633333,10.536667,13.141667,NaN,NaN,17.417404,NaN,NaN,12.812925,23.427673,NaN,10.608333,NaN,22.075758,NaN,NaN,15.651515,NaN,NaN,9.099099,2333.316667,1121.583333,14.279600,Male,8.708333,NaN,NaN,6.807018,NaN,9.344660,11.956790,16.675758,9.637255,15.304207,20.908163,17.450000,8.040741,9.678862,37.462963,21.466667,NaN,12.869369,12.778846,19.900000,8.688333,12.89047

In [121]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_mean_pb(neighbours)

13.052194985056378

In [122]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
neighbours = neighbours.filter(like='pb')
get_mean_pace(neighbours)

Chamonix to Delevret_pb               7.865632
Chamonix to La Charme_pb              4.107246
Chamonix to St-Gervais_pb             7.562468
Delevret to St-Gervais_pb             6.276170
La Charme to St-Gervais_pb           20.987963
St-Gervais to Contamines_pb           8.530636
Contamines to La Balme_pb            10.849829
La Balme to Bonhomme_pb              15.822677
Bonhomme to Chapieux_pb               8.413126
Chapieux to Col Seigne_pb            12.897654
Col Seigne to Lac Combal_pb           9.822732
Lac Combal to Mt-Favre_pb            16.748472
Mt-Favre to Checruit_pb               8.622861
Checruit to Courmayeur_pb             9.995866
Courmayeur to Courmayeur2_pb         33.638889
Courmayeur2 to Bertone_pb            19.520238
Courmayeur to Bertone_pb             22.935438
Bertone to Bonatti_pb                10.609428
Bonatti to Arnouvaz_pb               11.428846
Arnouvaz to Col Ferret_pb            21.159012
Col Ferret to La Fouly_pb             8.995056
La Fouly to C

In [123]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_npb(neighbours)

13.692201400190454

In [124]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_pb(neighbours)

10.331618736659898

In [125]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


7.562467700258399

In [126]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

10.702092736936772

## Calculate Predictions

In [127]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [128]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df_k5 = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df_k5 = df_k5.apply(lambda row: update_rows(row), axis=1)
df_k5 = df_k5[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace']]
df_k5

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace
0,14.183180,13.489501,10.702093,13.052195
1,15.188612,13.489501,9.452978,13.885782
2,16.367387,13.945114,9.752659,15.073215
3,17.602140,15.121991,11.190219,15.540614
4,14.964501,13.769848,10.702577,14.001582
5,16.136556,16.626208,11.789038,15.266217
6,14.533689,13.438194,10.714045,13.577574
7,16.813847,18.265522,9.754428,15.316651
8,15.182644,11.779689,9.451846,13.918248
9,16.945035,14.825623,11.757397,15.601512


In [129]:
df_k5 = comb.join(df_k5)

In [130]:
df_k5.to_csv('../../data/K_5_pace_predictions.csv')

# K = 20

## Define Functions

In [131]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(0)

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
40,2015,07293d6206c13d09a85044be9e4963fc,Male,Senior Men (23-39),8.903186,NaN,NaN,7.578947,NaN,10.453074,12.619342,17.436364,9.307190,12.944984,19.156463,18.104167,7.655556,9.682927,34.074074,20.391667,NaN,11.891892,13.105769,24.907407,10.778333,13.861905,NaN,NaN,20.306785,NaN,NaN,9.568027,23.845912,NaN,8.997222,NaN,22.961039,NaN,NaN,11.126263,NaN,NaN,6.846847,2308.916667,1039.666667,14.660214
83,2013,0de585e8522f24147edec7692c655feb,Male,Senior Men (23-39),9.654412,NaN,NaN,7.048246,NaN,11.682848,12.440329,18.636364,8.405229,15.037217,8.785714,22.533333,9.344444,10.142276,NaN,NaN,29.319728,12.677928,12.820513,25.881481,9.910000,13.589286,16.690236,NaN,NaN,13.068783,NaN,NaN,25.823899,NaN,9.702778,NaN,21.945887,NaN,NaN,13.378788,NaN,NaN,9.993243,2362.733333,1127.783333,14.521373
105,2011,11c24a3c9ee9fcc173d9c21fa29c2bee,Male,Senior Men (23-39),8.547794,NaN,NaN,7.914474,NaN,8.734628,10.627572,16.215152,9.258170,13.211974,9.673469,15.450000,11.303704,10.609756,NaN,NaN,25.843537,11.731982,12.285256,21.618519,14.138333,15.263095,NaN,24.518248,NaN,NaN,20.773504,NaN,18.229560,NaN,11.741667,NaN,NaN,11.129630,NaN,NaN,11.429119,NaN,NaN,2392.733333,1156.016667,13.923876
106,2008,11c24a3c9ee9fcc173d9c21fa29c2bee,Male,Senior Men (23-39),NaN,4.958696,NaN,NaN,8.022222,8.572816,10.600823,15.254545,9.849673,12.626214,9.139456,15.066667,NaN,18.711382,NaN,NaN,20.064626,11.299550,11.544872,18.477778,11.096667,13.010714,18.638047,NaN,NaN,19.174603,NaN,NaN,25.094340,NaN,15.813889,NaN,24.173160,NaN,NaN,17.696970,NaN,NaN,12.558559,2271.200000,1014.216667,14.410707
118,2009,138b90e5dfce79bb890a7c8393d22982,Male,Senior Men (23-39),NaN,NaN,9.158915,NaN,NaN,10.208738,12.047325,18.227273,11.359477,14.207120,8.238095,17.487500,8.911111,12.333333,NaN,NaN,23.272109,10.795045,11.310897,19.744444,11.431667,11.636905,16.858586,NaN,NaN,13.838624,NaN,NaN,20.877358,NaN,12.711111,NaN,22.694805,NaN,NaN,12.479798,NaN,NaN,8.990991,2247.433333,954.133333,13.861793
138,2016,16f90397fb6a11351cb330e03f8edc1b,Male,Senior Men (23-39),8.291667,NaN,NaN,6.407895,NaN,8.988673,10.921811,15.424242,8.460784,12.478964,18.404762,16.808333,9.644444,11.483740,20.796296,NaN,16.095238,13.918919,13.474359,19.929630,10.646667,12.414286,NaN,NaN,15.141593,NaN,NaN,12.843537,22.644654,NaN,13.088889,NaN,17.588745,NaN,NaN,15.888889,NaN,NaN,10.518018,2185.816667,865.783333,13.692201
150,2009,17d416171d9f6e3dab9a4a2561dd4575,Male,Senior Men (23-39),NaN,NaN,7.855814,NaN,NaN,8.398058,10.109053,14.390909,8.196078,12.108414,10.081633,21.837500,9.574074,10.048780,NaN,NaN,20.860544,16.756757,12.028846,20.237037,8.771667,11.932143,15.611111,NaN,NaN,14.126984,NaN,NaN,21.496855,NaN,11.733333,NaN,21.467532,NaN,N

In [132]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    #euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    #euclidean_distance = euclidean_distance.head(10)
    #euclidean_distance_1 = euclidean_distance.head(1)
    #euclidean_distance_2 = euclidean_distance.head(2)
    #euclidean_distance_5 = euclidean_distance.head(5)
    euclidean_distance_20 = euclidean_distance.head(20)
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(0)

1439     11.253756
664      16.170319
415      30.498519
1127     46.203162
1087     50.055875
276      53.417078
951      61.340387
83       66.482538
1284     70.494002
741      74.141372
665      75.397558
1336     77.681757
592      78.724871
40       78.811073
1458     79.467827
804      82.316050
672      86.193188
1036     87.035716
1050     87.281867
463      87.944371
152      94.632593
955      98.053782
177      99.026358
1060    100.612989
190     100.660517
106     104.855065
105     111.134709
537     112.555974
707     117.201509
1275    118.610154
769     119.282932
616     125.603123
979     132.449373
460     141.721499
118     164.646742
1508    165.978477
160     177.400300
1560    180.007351
1552    184.610146
615     185.937895
1053    189.519516
360     189.959389
1526    197.642647
1106    201.019217
1201    208.934575
653     209.866702
980     213.524088
1589    215.150893
1536    216.062354
982     218.705507
1528    226.085016
1163    233.130264
150     246.

In [133]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0).head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
1439,2014,de99417dced7762c0b3cebbd7e98055f,Male,Senior Men (23-39),7.985294,NaN,NaN,6.383772,NaN,10.182848,13.549383,15.912121,9.202614,15.025890,9.469388,21.008333,8.385185,11.955285,NaN,NaN,26.289116,11.592342,11.839744,23.585185,9.601667,15.147619,NaN,NaN,17.762537,NaN,NaN,14.472789,24.345912,NaN,12.725000,NaN,21.937229,NaN,NaN,14.883838,NaN,NaN,7.371622,2318.133333,1106.400000,14.192280,Male,7.661765,NaN,NaN,6.030702,NaN,9.414239,12.446502,14.700000,8.771242,12.987055,19.044218,21.841667,8.459259,9.768293,47.888889,NaN,15.438776,10.894144,11.660256,24.492593,8.551667,12.029762,NaN,NaN,18.497050,NaN,NaN,11.914966,19.415094,NaN,10.997222,NaN,18.071429,NaN,NaN,15.969697,NaN,NaN,9.500000,2188.550000,868.516667,14.657859
664,2014,67870601366b233f04bdab9d3eda22d1,Male,Senior Men (23-39),7.601716,NaN,NaN,6.221491,NaN,9.451456,13.199588,16.615152,8.620915,13.956311,9.176871,19.737500,11.248148,10.658537,NaN,NaN,22.479592,11.623874,14.586538,21.288889,11.016667,14.070238,NaN,NaN,17.457227,NaN,NaN,12.316327,23.462264,NaN,10.752778,NaN,26.049784,NaN,NaN,19.767677,NaN,NaN,11.119369,2321.816667,1110.083333,14.269954,Male,NaN,NaN,NaN,NaN,28.202778,8.752427,10.965021,15.639394,8.578431,12.710356,9.020408,16.545833,NaN,18.422764,NaN,NaN,18.843537,12.038288,10.910256,21.225926,11.085000,11.665476,16.653199,NaN,NaN,13.740741,NaN,NaN,25.688679,NaN,14.894444,NaN,24.712121,NaN,NaN,17.186869,NaN,NaN,10.042793,2200.883333,943.900000,15.342034
415,2014,40607767095ee0cb77a09a3c21e849a7,Male,Senior Men (23-39),9.354167,NaN,NaN,8.320175,NaN,10.438511,11.810700,18.436364,10.767974,15.687702,10.462585,22.250000,9.525926,10.817073,NaN,NaN,24.972789,11.885135,12.576923,20.633333,10.536667,13.141667,NaN,NaN,17.417404,NaN,NaN,12.812925,23.427673,NaN,10.608333,NaN,22.075758,NaN,NaN,15.651515,NaN,NaN,9.099099,2333.316667,1121.583333,14.279600,Male,8.708333,NaN,NaN,6.807018,NaN,9.344660,11.956790,16.675758,9.637255,15.304207,20.908163,17.450000,8.040741,9.678862,37.462963,21.466667,NaN,12.869369,12.778846,19.900000,8.688333,12.89047

In [134]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_mean_pb(neighbours)

13.052194985056378

In [135]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
neighbours = neighbours.filter(like='pb')
get_mean_pace(neighbours)

Chamonix to Delevret_pb               7.865632
Chamonix to La Charme_pb              4.107246
Chamonix to St-Gervais_pb             7.562468
Delevret to St-Gervais_pb             6.276170
La Charme to St-Gervais_pb           20.987963
St-Gervais to Contamines_pb           8.530636
Contamines to La Balme_pb            10.849829
La Balme to Bonhomme_pb              15.822677
Bonhomme to Chapieux_pb               8.413126
Chapieux to Col Seigne_pb            12.897654
Col Seigne to Lac Combal_pb           9.822732
Lac Combal to Mt-Favre_pb            16.748472
Mt-Favre to Checruit_pb               8.622861
Checruit to Courmayeur_pb             9.995866
Courmayeur to Courmayeur2_pb         33.638889
Courmayeur2 to Bertone_pb            19.520238
Courmayeur to Bertone_pb             22.935438
Bertone to Bonatti_pb                10.609428
Bonatti to Arnouvaz_pb               11.428846
Arnouvaz to Col Ferret_pb            21.159012
Col Ferret to La Fouly_pb             8.995056
La Fouly to C

In [136]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_npb(neighbours)

13.692201400190454

In [137]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_pb(neighbours)

10.331618736659898

In [138]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


7.562467700258399

In [139]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

10.702092736936772

## Calculate Predictions

In [140]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [142]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df_k20 = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df_k20 = df_k20.apply(lambda row: update_rows(row), axis=1)
df_k20 = df_k20[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace']]
df_k20

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace
0,14.183180,13.489501,10.702093,13.052195
1,15.188612,13.489501,9.452978,13.885782
2,16.367387,13.945114,9.752659,15.073215
3,17.602140,15.121991,11.190219,15.540614
4,14.964501,13.769848,10.702577,14.001582
5,16.136556,16.626208,11.789038,15.266217
6,14.533689,13.438194,10.714045,13.577574
7,16.813847,18.265522,9.754428,15.316651
8,15.182644,11.779689,9.451846,13.918248
9,16.945035,14.825623,11.757397,15.601512


In [143]:
df_k20 = comb.join(df_k20)

In [144]:
df_k20.to_csv('../../data/K_20_pace_predictions.csv')

# K = 1 

## Define Functions

In [145]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(0)

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
40,2015,07293d6206c13d09a85044be9e4963fc,Male,Senior Men (23-39),8.903186,NaN,NaN,7.578947,NaN,10.453074,12.619342,17.436364,9.307190,12.944984,19.156463,18.104167,7.655556,9.682927,34.074074,20.391667,NaN,11.891892,13.105769,24.907407,10.778333,13.861905,NaN,NaN,20.306785,NaN,NaN,9.568027,23.845912,NaN,8.997222,NaN,22.961039,NaN,NaN,11.126263,NaN,NaN,6.846847,2308.916667,1039.666667,14.660214
83,2013,0de585e8522f24147edec7692c655feb,Male,Senior Men (23-39),9.654412,NaN,NaN,7.048246,NaN,11.682848,12.440329,18.636364,8.405229,15.037217,8.785714,22.533333,9.344444,10.142276,NaN,NaN,29.319728,12.677928,12.820513,25.881481,9.910000,13.589286,16.690236,NaN,NaN,13.068783,NaN,NaN,25.823899,NaN,9.702778,NaN,21.945887,NaN,NaN,13.378788,NaN,NaN,9.993243,2362.733333,1127.783333,14.521373
105,2011,11c24a3c9ee9fcc173d9c21fa29c2bee,Male,Senior Men (23-39),8.547794,NaN,NaN,7.914474,NaN,8.734628,10.627572,16.215152,9.258170,13.211974,9.673469,15.450000,11.303704,10.609756,NaN,NaN,25.843537,11.731982,12.285256,21.618519,14.138333,15.263095,NaN,24.518248,NaN,NaN,20.773504,NaN,18.229560,NaN,11.741667,NaN,NaN,11.129630,NaN,NaN,11.429119,NaN,NaN,2392.733333,1156.016667,13.923876
106,2008,11c24a3c9ee9fcc173d9c21fa29c2bee,Male,Senior Men (23-39),NaN,4.958696,NaN,NaN,8.022222,8.572816,10.600823,15.254545,9.849673,12.626214,9.139456,15.066667,NaN,18.711382,NaN,NaN,20.064626,11.299550,11.544872,18.477778,11.096667,13.010714,18.638047,NaN,NaN,19.174603,NaN,NaN,25.094340,NaN,15.813889,NaN,24.173160,NaN,NaN,17.696970,NaN,NaN,12.558559,2271.200000,1014.216667,14.410707
118,2009,138b90e5dfce79bb890a7c8393d22982,Male,Senior Men (23-39),NaN,NaN,9.158915,NaN,NaN,10.208738,12.047325,18.227273,11.359477,14.207120,8.238095,17.487500,8.911111,12.333333,NaN,NaN,23.272109,10.795045,11.310897,19.744444,11.431667,11.636905,16.858586,NaN,NaN,13.838624,NaN,NaN,20.877358,NaN,12.711111,NaN,22.694805,NaN,NaN,12.479798,NaN,NaN,8.990991,2247.433333,954.133333,13.861793
138,2016,16f90397fb6a11351cb330e03f8edc1b,Male,Senior Men (23-39),8.291667,NaN,NaN,6.407895,NaN,8.988673,10.921811,15.424242,8.460784,12.478964,18.404762,16.808333,9.644444,11.483740,20.796296,NaN,16.095238,13.918919,13.474359,19.929630,10.646667,12.414286,NaN,NaN,15.141593,NaN,NaN,12.843537,22.644654,NaN,13.088889,NaN,17.588745,NaN,NaN,15.888889,NaN,NaN,10.518018,2185.816667,865.783333,13.692201
150,2009,17d416171d9f6e3dab9a4a2561dd4575,Male,Senior Men (23-39),NaN,NaN,7.855814,NaN,NaN,8.398058,10.109053,14.390909,8.196078,12.108414,10.081633,21.837500,9.574074,10.048780,NaN,NaN,20.860544,16.756757,12.028846,20.237037,8.771667,11.932143,15.611111,NaN,NaN,14.126984,NaN,NaN,21.496855,NaN,11.733333,NaN,21.467532,NaN,N

In [146]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    #euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    #euclidean_distance = euclidean_distance.head(10)
    euclidean_distance_1 = euclidean_distance.head(1)
    #euclidean_distance_2 = euclidean_distance.head(2)
    #euclidean_distance_5 = euclidean_distance.head(5)
    #euclidean_distance_20 = euclidean_distance.head(20)
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(0)

1439     11.253756
664      16.170319
415      30.498519
1127     46.203162
1087     50.055875
276      53.417078
951      61.340387
83       66.482538
1284     70.494002
741      74.141372
665      75.397558
1336     77.681757
592      78.724871
40       78.811073
1458     79.467827
804      82.316050
672      86.193188
1036     87.035716
1050     87.281867
463      87.944371
152      94.632593
955      98.053782
177      99.026358
1060    100.612989
190     100.660517
106     104.855065
105     111.134709
537     112.555974
707     117.201509
1275    118.610154
769     119.282932
616     125.603123
979     132.449373
460     141.721499
118     164.646742
1508    165.978477
160     177.400300
1560    180.007351
1552    184.610146
615     185.937895
1053    189.519516
360     189.959389
1526    197.642647
1106    201.019217
1201    208.934575
653     209.866702
980     213.524088
1589    215.150893
1536    216.062354
982     218.705507
1528    226.085016
1163    233.130264
150     246.

In [147]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0).head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
1439,2014,de99417dced7762c0b3cebbd7e98055f,Male,Senior Men (23-39),7.985294,NaN,NaN,6.383772,NaN,10.182848,13.549383,15.912121,9.202614,15.025890,9.469388,21.008333,8.385185,11.955285,NaN,NaN,26.289116,11.592342,11.839744,23.585185,9.601667,15.147619,NaN,NaN,17.762537,NaN,NaN,14.472789,24.345912,NaN,12.725000,NaN,21.937229,NaN,NaN,14.883838,NaN,NaN,7.371622,2318.133333,1106.400000,14.192280,Male,7.661765,NaN,NaN,6.030702,NaN,9.414239,12.446502,14.700000,8.771242,12.987055,19.044218,21.841667,8.459259,9.768293,47.888889,NaN,15.438776,10.894144,11.660256,24.492593,8.551667,12.029762,NaN,NaN,18.497050,NaN,NaN,11.914966,19.415094,NaN,10.997222,NaN,18.071429,NaN,NaN,15.969697,NaN,NaN,9.500000,2188.550000,868.516667,14.657859
664,2014,67870601366b233f04bdab9d3eda22d1,Male,Senior Men (23-39),7.601716,NaN,NaN,6.221491,NaN,9.451456,13.199588,16.615152,8.620915,13.956311,9.176871,19.737500,11.248148,10.658537,NaN,NaN,22.479592,11.623874,14.586538,21.288889,11.016667,14.070238,NaN,NaN,17.457227,NaN,NaN,12.316327,23.462264,NaN,10.752778,NaN,26.049784,NaN,NaN,19.767677,NaN,NaN,11.119369,2321.816667,1110.083333,14.269954,Male,NaN,NaN,NaN,NaN,28.202778,8.752427,10.965021,15.639394,8.578431,12.710356,9.020408,16.545833,NaN,18.422764,NaN,NaN,18.843537,12.038288,10.910256,21.225926,11.085000,11.665476,16.653199,NaN,NaN,13.740741,NaN,NaN,25.688679,NaN,14.894444,NaN,24.712121,NaN,NaN,17.186869,NaN,NaN,10.042793,2200.883333,943.900000,15.342034
415,2014,40607767095ee0cb77a09a3c21e849a7,Male,Senior Men (23-39),9.354167,NaN,NaN,8.320175,NaN,10.438511,11.810700,18.436364,10.767974,15.687702,10.462585,22.250000,9.525926,10.817073,NaN,NaN,24.972789,11.885135,12.576923,20.633333,10.536667,13.141667,NaN,NaN,17.417404,NaN,NaN,12.812925,23.427673,NaN,10.608333,NaN,22.075758,NaN,NaN,15.651515,NaN,NaN,9.099099,2333.316667,1121.583333,14.279600,Male,8.708333,NaN,NaN,6.807018,NaN,9.344660,11.956790,16.675758,9.637255,15.304207,20.908163,17.450000,8.040741,9.678862,37.462963,21.466667,NaN,12.869369,12.778846,19.900000,8.688333,12.89047

In [148]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_mean_pb(neighbours)

13.052194985056378

In [149]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
neighbours = neighbours.filter(like='pb')
get_mean_pace(neighbours)

Chamonix to Delevret_pb               7.865632
Chamonix to La Charme_pb              4.107246
Chamonix to St-Gervais_pb             7.562468
Delevret to St-Gervais_pb             6.276170
La Charme to St-Gervais_pb           20.987963
St-Gervais to Contamines_pb           8.530636
Contamines to La Balme_pb            10.849829
La Balme to Bonhomme_pb              15.822677
Bonhomme to Chapieux_pb               8.413126
Chapieux to Col Seigne_pb            12.897654
Col Seigne to Lac Combal_pb           9.822732
Lac Combal to Mt-Favre_pb            16.748472
Mt-Favre to Checruit_pb               8.622861
Checruit to Courmayeur_pb             9.995866
Courmayeur to Courmayeur2_pb         33.638889
Courmayeur2 to Bertone_pb            19.520238
Courmayeur to Bertone_pb             22.935438
Bertone to Bonatti_pb                10.609428
Bonatti to Arnouvaz_pb               11.428846
Arnouvaz to Col Ferret_pb            21.159012
Col Ferret to La Fouly_pb             8.995056
La Fouly to C

In [150]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_npb(neighbours)

13.692201400190454

In [151]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_pb(neighbours)

10.331618736659898

In [152]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


7.562467700258399

In [153]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

10.702092736936772

## Calculate Predictions

In [154]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [155]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df1 = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df1 = df1.apply(lambda row: update_rows(row), axis=1)
df1 = df1[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace']]
df1

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace
0,14.183180,13.489501,10.702093,13.052195
1,15.188612,13.489501,9.452978,13.885782
2,16.367387,13.945114,9.752659,15.073215
3,17.602140,15.121991,11.190219,15.540614
4,14.964501,13.769848,10.702577,14.001582
5,16.136556,16.626208,11.789038,15.266217
6,14.533689,13.438194,10.714045,13.577574
7,16.813847,18.265522,9.754428,15.316651
8,15.182644,11.779689,9.451846,13.918248
9,16.945035,14.825623,11.757397,15.601512


In [156]:
df1 = comb.join(df1)

In [157]:
df1.to_csv('../../data/K_1_pace_predictions.csv')